# Assignment 2: Convergence, Accounting, & Development

**GLBL 5010: Economics for Global Affairs**

**Group Members:**
* Kevin Chen
* Pranav Pattatathunaduvil
* Lucy Kim

---

# Data Cleaning

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf

# Choosing country code for later question
MY_COUNTRY_CODE = "USA"

# Load the data
df = pd.read_csv('hwk2_accounting.csv', sep='\t')

# Filter for rows that have data for both 1960 and 2018 to ensure consistent samples
df_clean = df.dropna(subset=['pop1960', 'cgdpo1960', 'cn1960', 'pop2018', 'cgdpo2018', 'cn2018']).copy()

print("Data loaded. Number of countries:", len(df_clean))
print(df_clean.head())

Data loaded. Number of countries: 102
  countrycode    country    pop1960    hc1960  cgdpo1960      cn1960  \
3         ARG  Argentina  20.545674  1.953866    65001.0  107963.000   
5         AUS  Australia  10.470019  2.746758   163815.0  744405.700   
6         AUT    Austria   7.093828  2.403941    71811.0  176051.700   
7         BDI    Burundi   2.781159  1.095495     1920.0    5961.864   
8         BEL    Belgium   9.113383  2.307354    94903.0  493083.300   

     pop2018    hc2018  cgdpo2018      cn2018                     region  
3  44.361150  3.065968    1022236  3361087.00  Latin America & Caribbean  
5  24.898152  3.536047    1350340  5795477.00        East Asia & Pacific  
6   8.891388  3.369997     470542  2835065.00      Europe & Central Asia  
7  11.175374  1.402834       9048    18111.23         Sub-Saharan Africa  
8  11.482178  3.142735     510223  3436160.00      Europe & Central Asia  
